In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, auc, plot_roc_curve, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold

In [2]:
inner_fold = 4
outer_fold = 5

In [3]:
data = pd.read_csv("./../result/data_created.csv")

In [4]:
groups = data.PID

In [5]:
X = data.loc[:, ["mvm", "sdvm", "df", "p625", "fpdf", "mangle", "sdangle"]]
y = data["group"]

## Use nested cross-validation 
## Outer CV train-validation on 8 participant, test on the other two
## Inner CV 8 participant in total, train on 6, validate on 2

In [11]:
# Logistic Regression
# configure the cross-validation procedure
inner_cv = GroupKFold(n_splits=inner_fold)
outer_cv = GroupKFold(n_splits=outer_fold)

model = Pipeline([('normalizer', StandardScaler()),  # normalize data
                  ('clf', LogisticRegression(random_state=1))  # fit Logistic regression model
])


params = {
    'clf__solver' : ['newton-cg', 'lbfgs', 'liblinear'],
    'clf__penalty' : ["l2"],
    'clf__C' : [100, 10, 1.0, 0.1, 0.01]
}

acc = []
f1 = []
precision = []
recall = []
auc_score = []


for train_index, test_index in outer_cv.split(X, y, groups=groups):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    grid = GridSearchCV(estimator=model,
                        param_grid=params,
                        cv=inner_cv,
                        scoring="accuracy",
                        refit=True,
                        verbose=1,
                        n_jobs=-1)
    
    grid.fit(x_train, y_train, groups=groups[train_index])
    prediction = grid.predict(x_test)
    
    _acc = accuracy_score(y_test, prediction)
    _f1 = f1_score(y_test, prediction)
    _precision = precision_score(y_test, prediction)
    _recall = recall_score(y_test, prediction)
    _fpr, _tpr, _thresholds = roc_curve(y_test, prediction, pos_label=1)
    _auc = auc(_fpr, _tpr)
    
    acc.append(_acc)
    f1.append(_f1)
    precision.append(_precision)
    recall.append(_recall)
    auc_score.append(_auc)

print("Average accuracy on test set: ", np.mean(acc))
print("Average F1-Score on test set: ", np.mean(f1))
print("Average precision on test set: ", np.mean(precision))
print("Average recall on test set: ", np.mean(recall))
print("Average auc on test set: ", np.mean(auc_score))

Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 15 candidates, totalling 60 fits


...

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.9s finished


Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Average accuracy on test set:  0.992
Average F1-Score on test set:  0.9879183673469388
Average precision on test set:  0.9766153846153847
Average recall on test set:  1.0
Average auc on test set:  0.9941176470588236


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.7s finished


In [8]:
# nested cross validation for Random Forest model
# configure the cross-validation procedure
inner_cv = GroupKFold(n_splits=inner_fold)
outer_cv = GroupKFold(n_splits=outer_fold)

# define the model
model = RandomForestClassifier(random_state=1)
params = {
    'n_estimators' : [50, 100, 150, 200],
    'max_features' : ["auto", "sqrt", "log2"],
    'min_samples_leaf' : [1, 3, 5]
}

acc = []
f1 = []
precision = []
recall = []
auc_score = []


for train_index, test_index in outer_cv.split(X, y, groups=groups):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    grid = GridSearchCV(estimator=model,
                        param_grid=params,
                        cv=inner_cv,
                        scoring="accuracy",
                        refit=True,
                        verbose=1,
                        n_jobs=-1)
    
    grid.fit(x_train, y_train, groups=groups[train_index])
    prediction = grid.predict(x_test)
    
    _acc = accuracy_score(y_test, prediction)
    _f1 = f1_score(y_test, prediction)
    _precision = precision_score(y_test, prediction)
    _recall = recall_score(y_test, prediction)
    _fpr, _tpr, _thresholds = roc_curve(y_test, prediction, pos_label=1)
    _auc = auc(_fpr, _tpr)
    
    acc.append(_acc)
    f1.append(_f1)
    precision.append(_precision)
    recall.append(_recall)
    auc_score.append(_auc)

print("Average accuracy on test set: ", np.mean(acc))
print("Average F1-Score on test set: ", np.mean(f1))
print("Average precision on test set: ", np.mean(precision))
print("Average recall on test set: ", np.mean(recall))
print("Average auc on test set: ", np.mean(auc_score))

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   31.1s finished


Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   30.6s finished


Fitting 4 folds for each of 36 candidates, totalling 144 fits


...

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   36.1s finished


Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:   32.4s finished


Average accuracy on test set:  0.992
Average F1-Score on test set:  0.9874077290490664
Average precision on test set:  0.992
Average recall on test set:  0.9833333333333334
Average auc on test set:  0.9897058823529411


In [9]:
# Support Vector Machine
# configure the cross-validation procedure
inner_cv = GroupKFold(n_splits=inner_fold)
outer_cv = GroupKFold(n_splits=outer_fold)

# define the model
model = Pipeline([('normalizer', StandardScaler()),  # normalize data
                  ('clf', SVC(random_state=1))  # fit Logistic regression model
])


params = {
    'clf__kernel' : ['poly', 'rbf', 'sigmoid'],
    'clf__gamma' : ["scale"],
    'clf__C' : [50, 10, 1.0, 0.1, 0.01]
}

acc = []
f1 = []
precision = []
recall = []
auc_score = []


for train_index, test_index in outer_cv.split(X, y, groups=groups):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    grid = GridSearchCV(estimator=model,
                        param_grid=params,
                        cv=inner_cv,
                        scoring="accuracy",
                        refit=True,
                        verbose=1,
                        n_jobs=-1)
    
    grid.fit(x_train, y_train, groups=groups[train_index])
    prediction = grid.predict(x_test)
    
    _acc = accuracy_score(y_test, prediction)
    _f1 = f1_score(y_test, prediction)
    _precision = precision_score(y_test, prediction)
    _recall = recall_score(y_test, prediction)
    _fpr, _tpr, _thresholds = roc_curve(y_test, prediction, pos_label=1)
    _auc = auc(_fpr, _tpr)
    
    acc.append(_acc)
    f1.append(_f1)
    precision.append(_precision)
    recall.append(_recall)
    auc_score.append(_auc)

print("Average accuracy on test set: ", np.mean(acc))
print("Average F1-Score on test set: ", np.mean(f1))
print("Average precision on test set: ", np.mean(precision))
print("Average recall on test set: ", np.mean(recall))
print("Average auc on test set: ", np.mean(auc_score))

Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  53 out of  60 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  53 out of  60 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 15 candidates, totalling 60 fits
Average accuracy on test set:  0.9973333333333333
Average F1-Score on test set:  0.9959183673469388
Average precision on test set:  0.992
Average recall on test set:  1.0
Average auc on test set:  0.9980392156862745


[Parallel(n_jobs=-1)]: Done  53 out of  60 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.4s finished


In [10]:
# Decision Tree
# configure the cross-validation procedure
inner_cv = GroupKFold(n_splits=inner_fold)
outer_cv = GroupKFold(n_splits=outer_fold)

# define the model
model = DecisionTreeClassifier(random_state=1)

params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [2,4,6,8,10,12],
    'min_samples_leaf' : [6, 8]
}

for train_index, test_index in outer_cv.split(X, y, groups=groups):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    grid = GridSearchCV(estimator=model,
                        param_grid=params,
                        cv=inner_cv,
                        scoring="accuracy",
                        refit=True,
                        verbose=1,
                        n_jobs=-1)
    
    grid.fit(x_train, y_train, groups=groups[train_index])
    prediction = grid.predict(x_test)
    
    _acc = accuracy_score(y_test, prediction)
    _f1 = f1_score(y_test, prediction)
    _precision = precision_score(y_test, prediction)
    _recall = recall_score(y_test, prediction)
    _fpr, _tpr, _thresholds = roc_curve(y_test, prediction, pos_label=1)
    _auc = auc(_fpr, _tpr)
    
    acc.append(_acc)
    f1.append(_f1)
    precision.append(_precision)
    recall.append(_recall)
    auc_score.append(_auc)

print("Average accuracy on test set: ", np.mean(acc))
print("Average F1-Score on test set: ", np.mean(f1))
print("Average precision on test set: ", np.mean(precision))
print("Average recall on test set: ", np.mean(recall))
print("Average auc on test set: ", np.mean(auc_score))

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 4 folds for each of 24 candidates, totalling 96 fits
Average accuracy on test set:  0.9933333333333334
Average F1-Score on test set:  0.9894428816855141
Average precision on test set:  0.9920000000000002
Average recall on test set:  0.9875
Average auc on test set:  0.9917892156862745


[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    0.6s finished
